# Rback To the Future

In [170]:
import subprocess
import yaml
def get_checks(gh_actions_file: str):
    with open(gh_actions_file) as f:
        actions = yaml.safe_load(f)
    steps = actions['jobs']['scavenger']['steps']
    checks = [step for step in steps if 'Checks' in step['name']]
    return checks

def run_checks(checks):
    for check in checks:
        cmd = f"cd /workspaces/intro-to-kube\n{checks[0]['run']}"
        subprocess.run(
            cmd, 
            capture_output=False, 
            shell=True, 
            executable='/bin/bash',
            cwd = '/workspaces/intro-to-kube/rbac-to-the-future'
        )
        
# run_checks('/workspaces/intro-to-kube/.github/workflows/pesky-persistent-db.yml')
checks = get_checks('/workspaces/intro-to-kube/.github/workflows/rbac-to-the-future.yml')

```bash
minikube start
bash setup.sh
```

In [168]:
!bash create_users.sh

Generating RSA private key, 2048 bit long modulus (2 primes)
...+++++
...............................................+++++
e is 65537 (0x010001)
Signature ok
subject=CN = Jesus, O = Jesus
Getting CA Private Key
Generating RSA private key, 2048 bit long modulus (2 primes)
.....................+++++
..........................................+++++
e is 65537 (0x010001)
Signature ok
subject=CN = Joey, O = Joey
Getting CA Private Key
Generating RSA private key, 2048 bit long modulus (2 primes)
.............+++++
.................................+++++
e is 65537 (0x010001)
Signature ok
subject=CN = Jessica, O = Jessica
Getting CA Private Key
Generating RSA private key, 2048 bit long modulus (2 primes)
...+++++
..........+++++
e is 65537 (0x010001)
Signature ok
subject=CN = Jules, O = Jules
Getting CA Private Key
User "Jesus" set.
Context "Jesus-context" modified.
User "Joey" set.
Context "Joey-context" modified.
User "Jessica" set.
Context "Jessica-context" modified.
User "Jules" set.
Contex

In [169]:
!kubectl config view

apiVersion: v1
clusters:
- cluster:
    certificate-authority: /home/codespace/.minikube/ca.crt
    extensions:
    - extension:
        last-update: Mon, 09 Oct 2023 14:13:37 UTC
        provider: minikube.sigs.k8s.io
        version: v1.31.2
      name: cluster_info
    server: https://192.168.49.2:8443
  name: minikube
contexts:
- context:
    cluster: minikube
    user: Jessica
  name: Jessica-context
- context:
    cluster: minikube
    user: Jesus
  name: Jesus-context
- context:
    cluster: minikube
    user: Joey
  name: Joey-context
- context:
    cluster: minikube
    user: Jules
  name: Jules-context
- context:
    cluster: minikube
    extensions:
    - extension:
        last-update: Mon, 09 Oct 2023 14:13:37 UTC
        provider: minikube.sigs.k8s.io
        version: v1.31.2
      name: context_info
    namespace: default
    user: minikube
  name: minikube
current-context: minikube
kind: Config
preferences: {}
users:
- name: Jessica
  user:
    client-certificate: /work

> Let's start with easiest Role and RoleBinding to build. Let's give read-only access to the Intern role by defining get, watch, and list actions to Pod and Deployment resources strictly to the dev namespace. Afterwards bind this Role to Jules via the RoleBinding.

### Jules / Intern

In [134]:
%%writefile roles/intern.yml
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: Intern
  namespace: dev
rules:
- apiGroups: [""] # "" indicates the core API group
  resources: ["pods", "deployments"]
  verbs: ["get", "watch", "list"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: InternBinding
  namespace: dev
subjects:
- kind: User
  name: Jules
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: Intern
  apiGroup: rbac.authorization.k8s.io

Overwriting roles/intern.yml


In [135]:
!kubectl apply -f roles/intern.yml

role.rbac.authorization.k8s.io/Intern created
rolebinding.rbac.authorization.k8s.io/InternBinding created


In [145]:
run_checks([checks[0]])

Switched to context "minikube".
role.rbac.authorization.k8s.io/Intern unchanged
rolebinding.rbac.authorization.k8s.io/InternBinding unchanged
Switched to context "Jules-context".
✅ Cannot access pods in non-dev namespace
✅ Cannot create/update resources in dev namespace
✅ Can access pods in dev namespace


Test if Jules has the right permissioning.

> The next Role to build will be DBAdmin. Here we want to grant the Database Admin the permissions they need to do any read/writes to any resources strictly within the database and backup namespaces. For this to happen, you will need to create a single Role, and then create multiple RoleBindings per namespaces for the user Jessica:

### Jessica / DBAdmin

In [147]:
%%writefile roles/db-admin.yml
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: DBAdmin
  namespace: database
rules:
- apiGroups: [""]
  resources: ["pods", "deployments"]
  verbs: ["get", "watch", "list", "create", "update", "delete"]
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: DBAdminBinding
  namespace: database
subjects:
- kind: User
  name: Jessica
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: DBAdmin
  apiGroup: rbac.authorization.k8s.io
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: DBAdmin
  namespace: backup
rules:
- apiGroups: [""]
  resources: ["pods", "deployments"]
  verbs: ["get", "watch", "list", "create", "update", "delete"]
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: DBAdminBinding
  namespace: backup
subjects:
- kind: User
  name: Jessica
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: DBAdmin
  apiGroup: rbac.authorization.k8s.io

Overwriting roles/db-admin.yml


In [148]:
%%bash
kubectl config use-context minikube
kubectl apply -f roles/db-admin.yml

Switched to context "minikube".


role.rbac.authorization.k8s.io/DBAdmin created
rolebinding.rbac.authorization.k8s.io/DBAdminBinding created
role.rbac.authorization.k8s.io/DBAdmin created
rolebinding.rbac.authorization.k8s.io/DBAdminBinding created


In [152]:
run_checks([checks[1]])

Switched to context "minikube".
role.rbac.authorization.k8s.io/DBAdmin unchanged
rolebinding.rbac.authorization.k8s.io/DBAdminBinding unchanged
role.rbac.authorization.k8s.io/DBAdmin unchanged
rolebinding.rbac.authorization.k8s.io/DBAdminBinding unchanged
NAMESPACES
------------------------
database
backup
------------------------
role.rbac.authorization.k8s.io/DBAdmin unchanged
rolebinding.rbac.authorization.k8s.io/DBAdminBinding unchanged
role.rbac.authorization.k8s.io/DBAdmin unchanged
rolebinding.rbac.authorization.k8s.io/DBAdminBinding unchanged
Switched to context "Jessica-context".
✅ Can access pods in database namespace
✅ Can access pods in backup namespace
✅ Can create pods in database namespace
✅ Can create pods in backup namespace
✅ Can delete resources in database namespace
✅ Can delete resources in backup namespace
✅ Cannot get resources in namespaces other than database or backup
✅ Cannot create/update resources in namespaces other than database or backup


### Joey / Developer

In [171]:
%%writefile roles/developer.yml
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: Developer
  namespace: dev
rules:
- apiGroups: [""]
  resources: ["pods", "deployments"]
  verbs: ["get", "watch", "list", "create", "update"]
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: DeveloperBinding
  namespace: dev
subjects:
- kind: User
  name: Joey
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: Developer
  apiGroup: rbac.authorization.k8s.io
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: Developer
  namespace: staging
rules:
- apiGroups: [""]
  resources: ["pods", "deployments"]
  verbs: ["get", "watch", "list", "create", "update"]
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: DeveloperBinding
  namespace: staging
subjects:
- kind: User
  name: Joey
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: Developer
  apiGroup: rbac.authorization.k8s.io
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: Developer
  namespace: prod
rules:
- apiGroups: [""]
  resources: ["pods", "deployments"]
  verbs: ["get", "watch", "list", "create", "update"]
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: DeveloperBinding
  namespace: prod
subjects:
- kind: User
  name: Joey
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: Developer
  apiGroup: rbac.authorization.k8s.io

Overwriting roles/developer.yml


In [172]:
%%bash
kubectl config use-context minikube
kubectl apply -f roles/developer.yml

Switched to context "minikube".


role.rbac.authorization.k8s.io/Developer created
rolebinding.rbac.authorization.k8s.io/DeveloperBinding created
role.rbac.authorization.k8s.io/Developer created
rolebinding.rbac.authorization.k8s.io/DeveloperBinding created
role.rbac.authorization.k8s.io/Developer created
rolebinding.rbac.authorization.k8s.io/DeveloperBinding created


In [173]:
run_checks([checks[2]])

Switched to context "minikube".
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
NAMESPACES
------------------------
dev
staging
prod
------------------------
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorization.k8s.io/DeveloperBinding unchanged
role.rbac.authorization.k8s.io/Developer unchanged
rolebinding.rbac.authorizat

```yaml
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: my-role
  namespace: my-namespace
rules:
- apiGroups: ["", "extensions", "apps"]
  resources: ["pods", "deployments"]
  verbs: ["get", "list", "watch", "create", "update", "delete"]
```

```yaml
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: my-role-binding
  namespace: my-namespace
subjects:
- kind: User
  name: my-user
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: my-role
  apiGroup: rbac.authorization.k8s.io
```

### Jesus / Kube Admin

>For our last role, we will work to build the KubeAdmin role. This role grants a lot more permissions, so instead of using the traditional Role and RoleBinding, we will define a ClusterRole and ClusterRoleBinding. This is perfect for non-namespaced resource management, which in our case, we hope that our KubeAdmin has all the necessary privileges in place to do any operation across all namespaces in our cluster. Grant this ClusterRoleBinding to Jesus.

In [175]:
%%writefile roles/kube-admin.yml
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: KubeAdmin
rules:
- apiGroups: [""]
  resources: ["pods", "deployments", "services", "configmaps", "secrets", "ingresses", "jobs", "cronjobs", "persistentvolumeclaims"]
  verbs: ["get", "watch", "list", "create", "update", "delete"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: KubeAdminClusterRoleBinding
subjects:
- kind: User
  name: Jesus
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: ClusterRole
  name: KubeAdmin
  apiGroup: rbac.authorization.k8s.io

Overwriting roles/kube-admin.yml


In [176]:
%%bash
kubectl config use-context minikube
kubectl apply -f roles/kube-admin.yml

Switched to context "minikube".
clusterrole.rbac.authorization.k8s.io/KubeAdmin created
clusterrolebinding.rbac.authorization.k8s.io/KubeAdminClusterRoleBinding created


In [177]:
run_checks([checks[3]])

Switched to context "minikube".
clusterrole.rbac.authorization.k8s.io/KubeAdmin unchanged
clusterrolebinding.rbac.authorization.k8s.io/KubeAdminClusterRoleBinding unchanged
Switched to context "Jesus-context".
✅ Can access pods in dev namespace
✅ Can access pods in staging namespace
✅ Can access pods in prod namespace
✅ Can access pods in database namespace
✅ Can access pods in backup namespace
✅ Can create pods in dev namespace
✅ Can create pods in staging namespace
✅ Can create pods in prod namespace
✅ Can create pods in database namespace
✅ Can create pods in backup namespace
✅ Can delete resources in dev namespace
✅ Can delete resources in staging namespace
✅ Can delete resources in prod namespace
✅ Can delete resources in database namespace
✅ Can delete resources in backup namespace


## Teardown

In [67]:
%%bash
minikube stop
minikube delete

* Stopping node "minikube"  ...
* Powering off "minikube" via SSH ...
* 1 node stopped.
* Deleting "minikube" in docker ...
* Deleting container "minikube" ...
* Removing /home/codespace/.minikube/machines/minikube ...
* Removed all traces of the "minikube" cluster.
